In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('test')\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/04 11:41:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.2.1'

In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-04 11:41:03--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.154.164
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.154.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv.1’

fhvhv_tripdata_2021 100%[===================>] 699.83M  36.2MB/s    in 20s     

2022-03-04 11:41:24 (34.7 MB/s) - ‘fhvhv_tripdata_2021-02.csv.1’ saved [733822658/733822658]



In [5]:
import pandas as pd

In [6]:
df_temp = pd.read_csv('fhvhv_tripdata_2021-02.csv')

In [7]:
spark.createDataFrame(df_temp).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [10]:
from pyspark.sql import types

In [12]:
schema = types.StructType([
    types.StructField('hvfhs_license_num',types.StringType(),True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.StringType(),True),
    types.StructField('dropoff_datetime',types.StringType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True)
])

In [13]:
df = spark.read\
    .option("header","true")\
    .schema(schema)\
    .csv('fhvhv_tripdata_2021-02.csv')

In [14]:
df = df.repartition(24)

In [15]:
df.write.parquet('fhvhv')

In [16]:
!ls -lh fhvhv

total 208M
-rw-r--r-- 1 kabachia kabachia    0 Mar  4 12:14 _SUCCESS
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00000-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00001-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00002-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00003-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00004-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00005-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00006-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.parquet
-rw-r--r-- 1 kabachia kabachia 8.7M Mar  4 12:14 part-00007-8ec01745-8ffe-452d-9bc2-c003cd8ade96-c000.snappy.par

In [19]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [20]:
df.registerTempTable('fhvhv_data')

/opt/spark/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [22]:
spark.sql("""
SELECT
    CAST(pickup_datetime AS DATE)
FROM 
    fhvhv_data
LIMIT 10
""").show()

+---------------+
|pickup_datetime|
+---------------+
|     2021-02-06|
|     2021-02-05|
|     2021-02-04|
|     2021-02-02|
|     2021-02-03|
|     2021-02-04|
|     2021-02-01|
|     2021-02-04|
|     2021-02-03|
|     2021-02-06|
+---------------+



In [23]:
spark.sql("""
SELECT
    count(1)
FROM 
    fhvhv_data
WHERE
    CAST(pickup_datetime AS DATE) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [26]:
spark.sql("""
SELECT
    pickup_datetime,
    dropoff_datetime,
    unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) duration
FROM 
    fhvhv_data
LIMIT 10
""").show()

+-------------------+-------------------+--------+
|    pickup_datetime|   dropoff_datetime|duration|
+-------------------+-------------------+--------+
|2021-02-05 14:23:17|2021-02-05 14:45:43|    1346|
|2021-02-01 06:02:37|2021-02-01 06:28:31|    1554|
|2021-02-01 16:42:47|2021-02-01 16:54:55|     728|
|2021-02-02 08:35:38|2021-02-02 08:52:48|    1030|
|2021-02-06 05:50:27|2021-02-06 06:20:08|    1781|
|2021-02-06 13:35:24|2021-02-06 14:06:37|    1873|
|2021-02-03 16:34:15|2021-02-03 16:57:57|    1422|
|2021-02-02 08:13:56|2021-02-02 08:25:53|     717|
|2021-02-04 18:03:52|2021-02-04 18:16:44|     772|
|2021-02-04 10:43:20|2021-02-04 11:00:28|    1028|
+-------------------+-------------------+--------+



In [28]:
spark.sql("""
SELECT
    CAST(pickup_datetime AS DATE) _date,
    MAX(unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) trip_duration 
FROM 
    fhvhv_data
GROUP BY
    _date
ORDER BY
    trip_duration DESC
""").show()

+----------+-------------+
|     _date|trip_duration|
+----------+-------------+
|2021-02-11|        75540|
|2021-02-17|        57221|
|2021-02-20|        44039|
|2021-02-03|        40653|
|2021-02-19|        37577|
|2021-02-25|        35010|
|2021-02-18|        34612|
|2021-02-10|        34169|
|2021-02-21|        32223|
|2021-02-09|        32087|
|2021-02-06|        31447|
|2021-02-02|        30913|
|2021-02-05|        30511|
|2021-02-12|        30148|
|2021-02-08|        30106|
|2021-02-14|        29777|
|2021-02-22|        28278|
|2021-02-27|        27170|
|2021-02-15|        25874|
|2021-02-04|        25592|
+----------+-------------+
only showing top 20 rows



In [29]:
spark.sql("""
SELECT
    COUNT(1) cnt,
    dispatching_base_num 
FROM 
    fhvhv_data
GROUP BY
    dispatching_base_num
ORDER BY
    cnt DESC
""").show()

+-------+--------------------+
|    cnt|dispatching_base_num|
+-------+--------------------+
|3233664|              B02510|
| 965568|              B02764|
| 882689|              B02872|
| 685390|              B02875|
| 559768|              B02765|
| 429720|              B02869|
| 322331|              B02887|
| 312364|              B02871|
| 311603|              B02864|
| 311089|              B02866|
| 305185|              B02878|
| 303255|              B02682|
| 274510|              B02617|
| 251617|              B02883|
| 244963|              B02884|
| 232173|              B02882|
| 215693|              B02876|
| 210137|              B02879|
| 200530|              B02867|
| 198938|              B02877|
+-------+--------------------+
only showing top 20 rows



In [30]:
df_zones = spark.read.parquet('zones')

In [32]:
df_zones.head()

Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR')

In [33]:
df_pickup_locations = df_zones.withColumnRenamed('Zone','pickup_Zone')

In [34]:
df_dropoff_locations = df_zones.withColumnRenamed('Zone','dropoff_Zone')

In [36]:
df_fhvhv_zoned = df\
    .join(df_pickup_locations, df.PULocationID==df_pickup_locations.LocationID)\
    .join(df_dropoff_locations, df.DOLocationID==df_dropoff_locations.LocationID)

In [38]:
df_fhvhv_zoned.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+----------+---------+--------------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID|  Borough|         pickup_Zone|service_zone|LocationID|  Borough|        dropoff_Zone|service_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+----------+---------+--------------------+------------+
|           HV0003|              B02877|2021-02-02 07:20:04|2021-02-02 07:29:56|          87|         249|   null|        87|Manhattan|Financial Distric...| Yellow Zone|       249|Manhattan|        West Village| Yellow Zone|
|           HV0005|              B02510|2021-02-05 22:31:58|2021-02-05 22:45:26|         241|       

In [39]:
from pyspark.sql import functions as F

In [41]:
df_fhvhv_zoned = df_fhvhv_zoned\
    .withColumn('LocationPair',F.concat(F.col('pickup_Zone'),F.lit('/'),F.col('dropoff_Zone')))

In [43]:
df_fhvhv_zoned.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+----------+---------+--------------------+------------+--------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID|  Borough|         pickup_Zone|service_zone|LocationID|  Borough|        dropoff_Zone|service_zone|        LocationPair|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+----------+---------+--------------------+------------+--------------------+
|           HV0003|              B02877|2021-02-02 07:20:04|2021-02-02 07:29:56|          87|         249|   null|        87|Manhattan|Financial Distric...| Yellow Zone|       249|Manhattan|        West Village| Yellow Zone|Financial Distric...|
|           HV00

In [49]:
pd.set_option('display.max_colwidth', None)

In [51]:
df_fhvhv_zoned.groupBy('LocationPair').count().orderBy("count",ascending=False).show(10,False)

+-----------------------------------------+-----+
|LocationPair                             |count|
+-----------------------------------------+-----+
|East New York/East New York              |45041|
|Borough Park/Borough Park                |37329|
|Canarsie/Canarsie                        |28026|
|Crown Heights North/Crown Heights North  |25976|
|Bay Ridge/Bay Ridge                      |17934|
|Jackson Heights/Jackson Heights          |14688|
|Astoria/Astoria                          |14688|
|Central Harlem North/Central Harlem North|14481|
|Bushwick South/Bushwick South            |14424|
|Flatbush/Ditmas Park/Flatbush/Ditmas Park|13976|
+-----------------------------------------+-----+
only showing top 10 rows

